In [ ]:
"""
Plotting simulation results

Authors: Milind Kumar Vaddiraju, ChatGPT, Copilot
"""

# Necessary imports
import copy
from datetime import datetime
import json
# %matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import os
import pickle
import sys

sys.path.insert(0, os.path.abspath('..'))

from network_classes import *
from utils import *


In [ ]:
# Some common functions

def return_folder_paths(experiment_foldername):
    folder_names = []
    for folder in os.listdir(experiment_foldername):
        # Check if the folder is a directory
        if os.path.isdir(experiment_foldername + folder + "/"):
            folder_names.append(experiment_foldername + folder + "/")
            
    return folder_names

def obtain_plot_information(experiment_key, plotting_information, data_key):
        y = []
        x = []

        results_allUEs_per_lambda_contention = plotting_information[experiment_key]["results_allUEs_per_lambda_contention"]
        for lambda_value in results_allUEs_per_lambda_contention:
                x.append(lambda_value)
                y.append(\
                       results_allUEs_per_lambda_contention[lambda_value][data_key])
                
        y = [i for _, i in sorted(zip(x,y))]
        x = sorted(x)

        return (x, y)


def extract_plotting_data(folder_names, key_accessor):
    plotting_data_temp = {}
    for folder in folder_names:
        experiment_filename = folder + "experiment_parameters.pkl"
        with open(experiment_filename, "rb") as file:
            data = pickle.load(file)
        key = key_accessor[folder]
        # print(key)
        if key in plotting_data_temp:
            plotting_data_temp[key]["lambda_range"] = np.concatenate((plotting_data_temp[key]["lambda_range"], data["experiment_parameters"]["lambda_range"]))
            plotting_data_temp[key]["results_allUEs_per_lambda_contention"].update(data["results_allUEs_per_lambda_contention"])
            # assert plotting_data_temp[key]["schedule_contention"] == data["schedule_contention"] TODO: Create equality for the Schedule class
            assert plotting_data_temp[key]["percentile_to_plot"] == data["experiment_parameters"]["percentile_to_plot"]
        else:
            plotting_data_temp[key] = {
                "lambda_range": data["experiment_parameters"]["lambda_range"],
                "results_allUEs_per_lambda_contention": data["results_allUEs_per_lambda_contention"],
                "schedule_contention": data["schedule_contention"],
                "percentile_to_plot": data["experiment_parameters"]["percentile_to_plot"],
                "num_UEs": data["experiment_parameters"]["num_UEs"]
            }
    return plotting_data_temp



In [ ]:
folder_names = [
                #"../results/simulation_4/2024_05_31_14_01_18/", 
                # "../results/simulation_4/2024_05_31_14_06_29/", 
                "../results/simulation_4/2024_05_31_14_10_05/", 
                "../results/simulation_4/2024_05_31_14_13_29/", 
                # "../results/simulation_4/2024_05_31_14_29_15/", 
                # "../results/simulation_4/2024_05_31_14_42_03/", 
                # "../results/simulation_4/2024_05_31_14_56_08/", 
                # "../results/simulation5/10UEs_dynamic_rr/2024_05_31_15_50_07/",
                "../results/simulation5/10UEs_dynamic_rr/2024_05_19_21_22_22/",
                # "../results/simulation6/2024_06_03_14_28_09/",
                "../results/simulation6/2024_06_03_21_36_17/",
                # "../results/simulation6/2024_06_03_21_40_31/",
                "../results/simulation6/2024_06_03_22_15_18/",
                "../results/simulation6/2024_06_03_22_26_24/",
                # "../results/simulation6/2024_06_03_22_33_29/",
                # "../results/simulation6/2024_06_04_12_47_09/",
                "../results/simulation6/2024_06_04_12_55_59/",
                # "../results/simulation6/2024_06_04_13_01_34/",
                # "../results/simulation6/2024_06_04_13_28_08/",
                "../results/simulation6/2024_06_04_13_39_22/",
                ]

key_accessor_values = {
    # "../results/simulation_4/2024_05_31_14_01_18/": "csma 0.1 per",
    # "../results/simulation_4/2024_05_31_14_06_29/": "rr 1500 0.1 per",
    "../results/simulation_4/2024_05_31_14_10_05/": "rr 1500 0 per",
    "../results/simulation_4/2024_05_31_14_13_29/": "csma 0 per",
    # "../results/simulation_4/2024_05_31_14_29_15/": "rr 800, 0 per",
    # "../results/simulation_4/2024_05_31_14_42_03/": "rr 800, 0.1 per",
    # "../results/simulation_4/2024_05_31_14_56_08/": "grr 2500, 0.1 per",
    # "../results/simulation5/10UEs_dynamic_rr/2024_05_31_15_50_07/" : "drr cwmax",
    "../results/simulation5/10UEs_dynamic_rr/2024_05_19_21_22_22/" : "drr cwmin",
    # "../results/simulation6/2024_06_03_14_28_09/" : "max weight 1000",
    "../results/simulation6/2024_06_03_21_36_17/" : "oldest first 1000",
    # "../results/simulation6/2024_06_03_21_40_31/" : "minimum latency 1000",
    "../results/simulation6/2024_06_03_22_15_18/" : "max weight dynamic",
    "../results/simulation6/2024_06_03_22_26_24/" : "oldest first dynamic",
    # "../results/simulation6/2024_06_03_22_33_29/" : "minimum latency dynamic",
    # "../results/simulation6/2024_06_04_12_47_09/" : "max weight 1000, sbr 505.4",
    "../results/simulation6/2024_06_04_12_55_59/" : "oldest first 1000, sbr 505.4",
    # "../results/simulation6/2024_06_04_13_01_34/" : "minimum latency 1000, sbr 505.4",
    # "../results/simulation6/2024_06_04_13_28_08/" : "max weight 1000, sbr 505.4, 3 period",
    "../results/simulation6/2024_06_04_13_39_22/" : "oldest first 1000, sbr 505.4, 3 period",
}

In [ ]:
label_prefix = {}
plotting_data = {}
plotting_keys = {}



plotting_data_temp = extract_plotting_data(folder_names, \
                                           key_accessor_values)


label_prefix["all"] = ""
plotting_data["all"] = plotting_data_temp



In [ ]:
plotting_keys["all"] = key_accessor_values.values()



save_file = False

scale = "linear"
percentile_to_plot = 99
percentile_filename = "percentile_latency_zoomed_" + scale + ".png"
percentile_slope_filename = "percentile_slope_zoomed_" + scale + ".png"
mean_filename = "mean_latency_zoomed_" + scale + ".png"
mean_slope_filename = "mean_slope_zoomed_" + scale + ".png"
n_packets_not_served_filename = "n_packets_not_served_zoomed_" + scale + ".png"
bus_occupancy_filename = "bus_occupancy_zoomed_" + scale + ".png"
n_wins_filename = "n_wins_zoomed_" + scale + ".png"
queue_slope_filename = "queue_slope_zoomed_" + scale + ".png"
# label_prefix_temp = "number of STAs = "

scaling_factor = 1e6


##### Generate color schemes

total_lines = sum(len(plotting_keys[schedule_key]) for schedule_key in label_prefix)

# Generate unique colors
colors = cm.get_cmap('tab20', total_lines)


##################### Plot the  percentile latency curve #####################
plot_linewidths = {}
plot_linewidths["dynamic_rr"] = 4
plot_linewidths["roundrobin_964B_80MHz"] = 4
line_index = 0

plt.figure(figsize=(10, 8))
for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        if schedule_key in plot_linewidths:
                linewidth = plot_linewidths[schedule_key]
        else:
                linewidth = 2
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "percentile_latency")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/1e3, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=linewidth, color=colors(line_index))
                line_index += 1


plt.xlabel("$\lambda$ (packets/s)", fontsize=15)
plt.ylabel("$99^{th}$ percentile latency (ms)", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(prop={'size': 10})
plt.grid()

if scale == "linear":
        plt.ylim(bottom=0)
        plt.ylim(top = 40)
        # plt.xlim(0,2000)
# plt.xlim(left=0)
# plt.ylim(0,50000)


if scale == "log":
        plt.yscale('log')
        plt.xscale('log')

title = ("$99^{th}$ percentile latency vs load\n" 
        )
plt.title(title, fontsize=18)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, percentile_filename))
plt.show()



##################### Plot the  mean latency curve #####################
line_index = 0
plt.figure(figsize=(10, 8))


for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        if schedule_key in plot_linewidths:
                linewidth = plot_linewidths[schedule_key]
        else:
                linewidth = 2
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "mean_latency")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/1e3, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=linewidth, color=colors(line_index))
                line_index += 1

plt.xlabel("lambda (packets/s)")
plt.ylabel("Mean latency (ms)")
plt.legend(prop={'size': 10})
plt.grid()
plt.ylim(0,40)
# plt.xlim(0,1000)


if scale == "log":
        plt.yscale('log')

title = (f"Mean latency vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, mean_filename))
plt.show()



##################### Plot the number of unserved packets #####################
line_index = 0
plt.figure(figsize=(10, 8))

for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "n_packets_not_served")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/plotting_data[schedule_key][plot_key]["num_UEs"], ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=2, color=colors(line_index))
                line_index += 1
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("Unserved packets")
plt.legend(prop={'size': 10})
plt.grid()

if scale == "log":
        plt.yscale('log')

title = (f"Unserved packets vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, n_packets_not_served_filename))
plt.show()